 # Flipkart Price Tracker and Email Notification System

In [ ]:
# Objective:-
#     In this project, I developed a Flipkart price tracker and email notification system using Python and its modules.
#     The objective was to showcase my Python programming skills and proficiency in utilizing relevant modules for web 
#     scraping, data manipulation, and email automation.
    
# Data Collection and Storage:-
#     Utilized web scraping techniques with libraries like BeautifulSoup and Requests to extract product details such as name 
#     and price from Flipkart's product pages. 
#     Implemented an automated data collection mechanism to periodically fetch the  latest prices and store them in a file.
#     Created a well-structured file format, such as CSV, to store the product details along with the corresponding prices.
    
    
# Data Processing and Comparison:-
#     Developed a Python script to read the data from website and stored them in file .
#     Utilized data manipulation techniques using modules like pandas to perform efficient and effective price comparisons.
#     Implemented algorithms to identify when the latest price is lower than the previous prices.
    
    
# Scheduling task and Email Notification System:-
#     Set up an automation mechanism,  Task Scheduler on Windows, to run the price tracking and email notification script at regular intervals.
#     Integrated the project with the email functionality to send notifications when the latest price is lower than the 
#     previous prices.   
    
# Error Handling and Robustness:-
#     Implemented error handling techniques to handle exceptions and errors that may occur during web scraping, data processing, or email sending.
#     Ensured the system's robustness by handling edge cases, such as network connectivity issues, invalid data, or email delivery failures.
    

# Testing and Validation:
#     Performed rigorous testing to ensure the accuracy and correctness of the price tracking and email notification system.
#     Conducted validation tests using sample data and edge cases to verify the system's functionality and reliability.
    


In [ ]:


def sendmail(url,title,price):
    try:
        email_sender = 'kumarbiswanath2030@gmail.com'
        email_password = '*******'
        email_receiver = '************'

        subject='price drop on flipkart'
        body = f'price for {title} has droped to {price}. to purchase pls go to {url} '

        em = EmailMessage()
        em['From'] = email_sender
        em['To'] = email_receiver
        em['Subject'] = subject
        em.set_content(body)
        context = ssl.create_default_context()

        with smtplib.SMTP_SSL('smtp.gmail.com', '465', context=context) as smtp:
            smtp.login(email_sender, email_password)
            smtp.sendmail(email_sender, email_receiver, em.as_string())
            
            
            
    except Exception as e:
        print(e)

In [ ]:
def scrap_web(url):
    max_retry=3
    retry_in=5
    for attempt in range(max_retry):
        try:
            data=requests.get(url)
            if data.status_code==200:
                #successful responce
                return data
            else:
                raise Exception(f'Received non-200 status code: {data.status_code}')
        
        
        except Exception as e:
            print(f"Network Error: {e}")
            if attempt < max_retry - 1:
                print(f"Retrying after {retry_in} seconds...")
                time.sleep(retry_in)
            else:
                print("Exceeded maximum number of retries.")
                return None
                
        

In [ ]:

def check_price():
    url='https://www.flipkart.com/apple-iphone-13-midnight-128-gb/p/itmca361aab1c5b0?pid=MOBG6VF5Q82T3XRS&lid=LSTMOBG6VF5Q82T3XRSOXJLM9&marketplace=FLIPKART&q=iphones&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=b1a222d8-69a7-4e6a-b5fb-a18f3ea4bd73.MOBG6VF5Q82T3XRS.SEARCH&ppt=hp&ppn=homepage&ssid=g2d76338680000001688888680290&qH=3e7fa8c51e2e4986'
    data=scrap_web(url)
        
        
    if data is not None:
        soup1=bs(data.content,"html.parser")

        title = soup1.find('span',class_='B_NuCI').text
        title=title.strip()

        price = soup1.find('div',class_='_30jeq3 _16Jk6d').get_text()
        price=price.replace(',','').strip()[1:]
        print(price)
        print(title)

        header=['date','title','price']
        rec=[today,title,price]
        #rec=[today,'its manual tshirt','999']
        if not os.path.exists('FlipkartWebScraperDataset.csv'):
            print('file not exist yet')
            pass
        else:
            print('file exists now')
            df=pd.read_csv('FlipkartWebScraperDataset.csv')
            if df['price'].min()>int(price):
                print('price has droped now')
                sendmail(url,title,price) 
                print('email sent successfully')

        with open('FlipkartWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
            print('csv writing started')
            writer = csv.writer(f)
            if os.stat('FlipkartWebScraperDataset.csv').st_size == 0:
                writer.writerow(header)
            writer.writerow(rec)
            print('writing completed')   
                
    else:
        print('failed to scrape website')



In [ ]:
import csv
import pandas as pd
import os
import time
import datetime
import smtplib
import requests
from bs4 import BeautifulSoup as bs
from email.message import EmailMessage
import ssl
import smtplib

check_price()
